# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import easydict
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
import sys
import albumentations as A
import albumentations.pytorch

from torch.utils.data import DataLoader, random_split, TensorDataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
from tqdm import tqdm
from torch import Tensor
from typing import Optional
from torch.nn.modules.transformer import _get_activation_fn
from datetime import datetime
from pathlib import Path
from albumentations.pytorch import ToTensorV2

# 데이터 불러오기 및 압축풀기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/sw해커톤/data_2.zip" "data_2.zip"
!unzip "data_2.zip"



Archive:  data_2.zip
  inflating: dirty_mnist_2nd.zip     

In [ ]:
!mkdir "./dirty_mnist"
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
!mkdir "./test_dirty_mnist"
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

args = easydict.EasyDict({
    "image_path": "./dirty_mnist/",
    "label_path": "/content/dirty_mnist_2nd_answer.csv",
    "kfold_idx": 0, ## 0~4까지 바꿔 가면서 5번 학습. 
    "epochs": 1000,
    "batch_size": 64
    "lr": 1e-3,
    "patience": 20,
    "resume": None,
    "device": device,
    "comments": None,
})

# 전처리 및 학습

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self, image_folder, label_df, transforms):        
        self.image_folder = image_folder   
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)
    
    def __getitem__(self, index):        
        image_fn = self.image_folder +\
            str(self.label_df.iloc[index,0]).zfill(5) + '.png'
                                              
        image = cv2.imread(image_fn, cv2.IMREAD_GRAYSCALE)        
        image = image.reshape([256, 256, 1])

        label = self.label_df.iloc[index,1:].values.astype('float')

        if self.transforms:            
            image = self.transforms(image)

        return image, label

basic_transform = T.Compose([
    T.ToTensor(),
    T.Lambda(lambda x: torch.cat([x, x, x], 0)), 
])    
    
mnist_transforms = {
    'train' : albumentations.Compose([
            albumentations.RandomRotate90(),
            albumentations.OneOf([
                albumentations.GridDistortion(distort_limit=(-0.3, 0.3), border_mode=cv2.BORDER_CONSTANT, p=1),
                albumentations.ShiftScaleRotate(rotate_limit=15, border_mode=cv2.BORDER_CONSTANT, p=1),        
                albumentations.ElasticTransform(alpha_affine=10, border_mode=cv2.BORDER_CONSTANT, p=1),
            ], p=1),    
            albumentations.Cutout(num_holes=3, max_h_size=8, max_w_size=8, fill_value=0),
            albumentations.pytorch.ToTensorV2(),
        ]),
    'valid' : albumentations.Compose([        
        albumentations.pytorch.ToTensorV2(),
        ]),
    'test' : albumentations.Compose([        
        albumentations.pytorch.ToTensorV2(),
        ]),
}

def train(train_loader, model, loss_func, device, optimizer, scheduler=None):
    n = 0
    running_loss = 0.0
    running_corrects = 0

    epoch_loss = 0.0
    epoch_acc = 0.0

    model.train()    

    with tqdm(train_loader, total=len(train_loader), desc="Train", file=sys.stdout) as iterator:
        for train_x, train_y in iterator:
            
            train_x = torch.stack([x.permute(1,2,0) for x in train_x])
            train_x = train_x.numpy()
            train_x = torch.stack([mnist_transforms["train"](image=k)["image"] for k in train_x])
            
            train_x = train_x.float().to(device)
            train_y = train_y.float().to(device)
            
            output = model(train_x)
            
            loss = loss_func(output, train_y)
            
            n += train_x.size(0)
            running_loss += loss.item() * train_x.size(0)

            epoch_loss = running_loss / float(n)

            output = output > 0.5
            running_corrects += (output == train_y).sum()
            epoch_acc = running_corrects / train_y.size(1) / n

            log = 'loss - {:.5f}, acc - {:.5f}'.format(epoch_loss, epoch_acc)
            
            iterator.set_postfix_str(log)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

    if scheduler:
        scheduler.step(epoch_loss)

    return epoch_loss, epoch_acc


def validate(valid_loader, model, loss_func, device, scheduler=None):
    n = 0
    running_loss = 0.0
    running_corrects = 0

    epoch_loss = 0.0
    epoch_acc = 0.0

    model.eval()

    with tqdm(valid_loader, total=len(valid_loader), desc="Valid", file=sys.stdout) as iterator:
        for train_x, train_y in iterator:
            train_x = train_x.float().to(device)
            train_y = train_y.float().to(device)

            with torch.no_grad():
                output = model(train_x)
            
            loss = loss_func(output, train_y)

            n += train_x.size(0)
            running_loss += loss.item() * train_x.size(0)

            epoch_loss = running_loss / float(n)

            output = output > 0.5
            running_corrects += (output == train_y).sum()
            epoch_acc = running_corrects / train_y.size(1) / n

            log = 'loss - {:.5f}, acc - {:.5f}'.format(epoch_loss, epoch_acc)

            iterator.set_postfix_str(log)

    if scheduler:
        scheduler.step(epoch_loss)

    return epoch_loss, epoch_acc

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/dropout/cutout.py:52: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,


In [ ]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        
        super().__init__()
        self.features = models.efficientnet_v2_m(weights="DEFAULT")
        #self.gap = nn.AdaptiveAvgPool2d(1)
        self.fn1 = nn.Linear(1000, 128)
        self.fn2 = nn.Linear(128, n_classes)
             
    def forward(self, x):
        x = self.features(x)
        #x = self.gap(x).squeeze(3).squeeze(2)
        #print(x.shape)
        x = self.fn1(x)
        x = F.relu(x)
        x = self.fn2(x)

        return x

In [ ]:
!head -n 3 /proc/meminfo

MemTotal:       53477460 kB
MemFree:        16159252 kB
MemAvailable:   51305280 kB


In [ ]:
print('=' * 50)
print('[info msg] arguments\n')
for key, value in vars(args).items():
    print(key, ":", value)
print('=' * 50)

assert os.path.isdir(args.image_path), 'wrong path'
assert os.path.isfile(args.label_path), 'wrong path'
if (args.resume):
    assert os.path.isfile(args.resume), 'wrong path'
assert args.kfold_idx < 5

seed_everything(777)

data_set = pd.read_csv(args.label_path)
valid_idx_nb = int(len(data_set) * (1 / 5))
valid_idx = np.arange(valid_idx_nb * args.kfold_idx, valid_idx_nb * (args.kfold_idx + 1))

print('[info msg] validation fold idx !!\n')        
print(valid_idx)
print('=' * 50)

train_data = data_set.drop(valid_idx)
valid_data = data_set.iloc[valid_idx]

train_set = DatasetMNIST(
    image_folder=args.image_path,
    label_df=train_data,
    transforms=basic_transform
)

valid_set = DatasetMNIST(
    image_folder=args.image_path,
    label_df=valid_data,
    transforms=basic_transform
)

train_data_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=args.batch_size,
        shuffle=True,
    )

valid_data_loader = torch.utils.data.DataLoader(
        valid_set,
        batch_size=args.batch_size,
        shuffle=False,
    )

model = CNN(26)

#if(args.resume):
#    model = EfficientNet.from_name(args.model, in_channels=1, num_classes=26, dropout_rate=0.5)
#    model.load_state_dict(torch.load(args.resume))
#    print('[info msg] pre-trained weight is loaded !!\n')        
#    print(args.resume)
#    print('=' * 50)

#else:
#    print('[info msg] {} model is created\n'.format(args.model))
#    model = EfficientNet.from_pretrained(args.model, in_channels=1, num_classes=26, dropout_rate=0.5, advprop=True)
#    print('=' * 50)

#if args.device == 'cuda' and torch.cuda.device_count() > 1 :
#    model = torch.nn.DataParallel(model)

model.to(args.device)

optimizer = torch.optim.Adam(model.parameters(), args.lr)
criterion = torch.nn.MultiLabelSoftMarginLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    mode='min',
    patience=7,
    factor=0.1,
    min_lr=1e-6,
    verbose=True
)

train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

best_loss = float("inf")

patience = 0

date_time = datetime.now().strftime("%m%d%H%M%S")
SAVE_DIR = os.path.join('/content/drive/MyDrive/sw해커톤/m/save2', date_time)

print('[info msg] training start !!\n')
startTime = datetime.now()
for epoch in range(args.epochs):        
    print('Epoch {}/{}'.format(epoch+1, args.epochs))
    train_epoch_loss, train_epoch_acc = train(
        train_loader=train_data_loader,
        model=model,
        loss_func=criterion,
        device=args.device,
        optimizer=optimizer,
        )
    train_loss.append(train_epoch_loss)
    train_acc.append(train_epoch_acc)

    valid_epoch_loss, valid_epoch_acc = validate(
        valid_loader=valid_data_loader,
        model=model,
        loss_func=criterion,
        device=args.device,
        scheduler=scheduler,
        )
    valid_loss.append(valid_epoch_loss)        
    valid_acc.append(valid_epoch_acc)

    if best_loss > valid_epoch_loss:
        patience = 0
        best_loss = valid_epoch_loss

        Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'model_best.pth.tar'))
        print('MODEL IS SAVED TO {}!!!'.format(date_time))
        
    else:
        patience += 1
        if patience > args.patience - 1:
            print('=======' * 10)
            print("[Info message] Early stopper is activated")
            break

elapsed_time = datetime.now() - startTime

train_loss = np.array(train_loss)
train_acc = np.array(train_acc)
valid_loss = np.array(valid_loss)
valid_acc = np.array(valid_acc)

best_loss_pos = np.argmin(valid_loss)

print('=' * 50)
print('[info msg] training is done\n')
print("Time taken: {}".format(elapsed_time))
print("best loss is {} w/ acc {} at epoch : {}".format(best_loss, valid_acc[best_loss_pos], best_loss_pos))    

print('=' * 50)
print('[info msg] {} model weight and log is save to {}\n'.format(args.model, SAVE_DIR))

with open(os.path.join(SAVE_DIR, 'log.txt'), 'w') as f:
    for key, value in vars(args).items():
        f.write('{} : {}\n'.format(key, value))            

    f.write('\n')
    f.write('total ecpochs : {}\n'.format(str(train_loss.shape[0])))
    f.write('time taken : {}\n'.format(str(elapsed_time)))
    f.write('best_train_loss {} w/ acc {} at epoch : {}\n'.format(np.min(train_loss), train_acc[np.argmin(train_loss)], np.argmin(train_loss)))
    f.write('best_valid_loss {} w/ acc {} at epoch : {}\n'.format(np.min(valid_loss), valid_acc[np.argmin(valid_loss)], np.argmin(valid_loss)))

plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='train loss')
plt.plot(valid_loss, 'o', label='valid loss')
plt.axvline(x=best_loss_pos, color='r', linestyle='--', linewidth=1.5)
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_acc, label='train acc')
plt.plot(valid_acc, 'o', label='valid acc')
plt.axvline(x=best_loss_pos, color='r', linestyle='--', linewidth=1.5)
plt.legend()
plt.savefig(os.path.join(SAVE_DIR, 'history.png'))

# Inference

In [ ]:
#import os
import cv2
from tqdm import tqdm
import pandas as pd
import argparse
import ttach as tta
from glob import glob
from datetime import datetime
import copy
import torch
from efficientnet_pytorch import EfficientNet
import albumentations.pytorch
import torchvision.transforms as T


class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self, image_folder, label_df, transforms):        
        self.image_folder = image_folder   
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)
    
    def __getitem__(self, index):        
        image_fn = self.image_folder +\
            str(self.label_df.iloc[index,0]).zfill(5) + '.png'
                                              
        image = cv2.imread(image_fn, cv2.IMREAD_GRAYSCALE)        
        image = image.reshape([256, 256, 1])

        label = self.label_df.iloc[index,1:].values.astype('float')

        if self.transforms:            
            image = self.transforms(image=image)['image'] / 255.0

        return image, label

base_transforms = {
    'test' : albumentations.Compose([        
        albumentations.pytorch.ToTensorV2(),
        ]),
}

#base_transforms = basic_transform = T.Compose([
#    T.ToTensor(),
    #T.Lambda(lambda x: torch.cat([x, x, x], 0)), 
#])

#tta_transforms = tta.Compose(
#    [
#        tta.Rotate90(angles=[0, 90, 180, 270]),
#    ]
#)

tta_transform = albumentations.Compose([
            albumentations.RandomRotate90(),
            albumentations.OneOf([
                albumentations.GridDistortion(distort_limit=(-0.3, 0.3), border_mode=cv2.BORDER_CONSTANT, p=1),
                albumentations.ShiftScaleRotate(rotate_limit=15, border_mode=cv2.BORDER_CONSTANT, p=1),        
                albumentations.ElasticTransform(alpha_affine=10, border_mode=cv2.BORDER_CONSTANT, p=1),
            ], p=1),    
            albumentations.Cutout(num_holes=16, max_h_size=15, max_w_size=15, fill_value=0),
            albumentations.pytorch.ToTensorV2(),
        ])

def main():
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    args = easydict.EasyDict({
    "image_path": "C://dacon/kw/test_dirty_mnist_2nd/",
    "label_path": "C://dacon/kw/sample_submission.csv",
    "weight_path": "C:/Users/woghs/2022_AI/save/efficientb3_baseline/",
    "out_path": "C://dacon/kw/",
    "model": "efficientnet-b3",
    "batch_size": 1,
    "device": device,
})
    
#    parser = argparse.ArgumentParser()
#    parser.add_argument('--image_path', type=str, default="./data/")
#    parser.add_argument('--label_path', type=str, default="./input/sample_submission.csv")
#    parser.add_argument('--weight_path', type=str, default='./input/model')
#    parser.add_argument('--out_path', type=str, default='./output/')

#    parser.add_argument('--model', type=str, default='efficientnet-b8')    
#    parser.add_argument('--batch_size', type=int, default=4)

#    parser.add_argument('--device', type=str, default=device)

#    args = parser.parse_args()

    assert os.path.isdir(args.image_path), 'wrong path'
    assert os.path.isfile(args.label_path), 'wrong path'
    assert os.path.isdir(args.weight_path), 'wrong path' 
    assert os.path.isdir(args.out_path), 'wrong path'

    print('=' * 50)
    print('[info msg] arguments')
    for key, value in vars(args).items():
        print(key, ":", value)
    
    weights = glob(os.path.join(args.weight_path, '*.tar'))

    test_df = pd.read_csv(args.label_path)

    test_set = DatasetMNIST(
        image_folder=args.image_path,
        label_df=test_df,
        transforms=base_transforms['test']
    )

    submission_df = copy.copy(test_df)

    for weight in weights:   
        model = EfficientNet.from_name(args.model, in_channels=1, num_classes=26)
        model.load_state_dict(torch.load(weight, map_location=args.device))
        print('=' * 50)
        print('[info msg] weight {} is loaded'.format(weight))

        test_data_loader = torch.utils.data.DataLoader(
                test_set,
                batch_size = args.batch_size,
                shuffle = False,
            )

        model.to(args.device)
        model.eval()
        #tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)

        batch_size = args.batch_size
        batch_index = 0

        print('=' * 50)
        print('[info msg] inference start')

        for i, (images, _) in enumerate(tqdm(test_data_loader)):
            sum_sub = model(images.cuda()) #원본
            #print(sum_sub)
            
            images = images.permute(0,2,3,1).numpy()
            
            for j in range(9): #변형 개수 및 합치는 과정
                sub_image = tta_transform(image=images.squeeze(0))["image"].unsqueeze(0)
                output = model(sub_image.float().cuda())
                sum_sub = sum_sub + output
            
            sum_final = sum_sub/10 #평균 soft voting
            
            #print(sum_final)
            
            #images = torch.stack([tta_transforms(image=k)["image"] for k in images])
            #images = images.float().to(args.device)
            
            #outputs = tta_model(images).detach().cpu().numpy().squeeze() # soft
            outputs = sum_final.detach().cpu().numpy().squeeze()
            #print(outputs)
            
            #outputs = (outputs > 0.5).astype(int) # hard vote
            batch_index = i * batch_size
            submission_df.iloc[batch_index:batch_index+batch_size, 1:] += outputs
    
    submission_df.iloc[:,1:] = (submission_df.iloc[:,1:] / len(weights) > 0.5).astype(int)
    
    SAVE_FN = os.path.join(args.out_path, datetime.now().strftime("%m%d%H%M") + '_please_submission.csv')

    submission_df.to_csv(
        SAVE_FN,
        index=False
        )

    print('=' * 50)
    print('[info msg] submission fils is saved to {}'.format(SAVE_FN))


if __name__ == '__main__':
    main()